In [4]:
library(raster)
library(terra)
library(sf)


Linking to GEOS 3.12.1, GDAL 3.9.0, PROJ 9.4.0; sf_use_s2() is TRUE



In [11]:
#======================================================
# Working with multipart zip files in unix is annoying
#======================================================
# zip -FF gSSURGO_CONUS.zip --out gSSURGO_CONUS_combined_fix.zip
# unzip gSSURGO_CONUS_combined_fix.zip

st_layers(dsn = "/data-store/iplant/home/rjramos/gssurgo/October 2023 gSSURGO CONUS/gSSURGO_CONUS.gdb/")

Cannot open data source /data-store/iplant/home/rjramos/gssurgo/October 2023 gSSURGO CONUS/gSSURGO_CONUS.gdb


ERROR: Error in eval(expr, envir, enclos): Open failed.



In [ ]:
ogrListLayers("/data-store/iplant/home/rjramos/gssurgo/October 2023 gSSURGO CONUS/gSSURGO_CONUS.gbd")


f <- "/data-stonlcd_agg.grd"
r <- raster(f)

library(terra)
x <- rast(r)

# get the attributes
lev <- levels(r)[[1]]  
lev <- lev[, c("ID", "Land.Cover.Class")]
lev[,2] <- as.character(lev[,2])

x <- rast(r)
levels(x) <- lev
is.factor(x)